### Como manipular XML com XPath

In [1]:
from gensim.models import KeyedVectors
import unicodedata

import gzip
import xml.etree.ElementTree as ET

with gzip.open('pordesc2018-small.xml.gz') as pordesc2018:
    tree = ET.parse(pordesc2018)
    

### Monta Dicionario

In [126]:
from unicodedata import normalize
def remover_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

In [127]:
remover_acentos(t.find('./String').text)

'Deficiencia de Fibrinogenio'

In [135]:
wordModel = KeyedVectors.load_word2vec_format('health_w2v_unigram_50.bin', binary=True)

dictMesh = {}
for d in tree.findall("./DescriptorRecord"):
    terms = []
    
    for c in d.findall('.ConceptList/'):
        if c.find('./ScopeNote') != None:
            scope = c.find('./ScopeNote').text.replace('\n','').strip()
        for t in c.findall('./TermList/'):
            terms.append(t.find('./String').text)
            
            palavra_similar = []
            
            if t.find('./String').text.lower() in wordModel.vocab:
                sem_assento = remover_acentos(t.find('./String').text)
                palavra_similar = wordModel.most_similar_cosmul(sem_assento.lower(),topn=10)
                for similar, porcentagem in palavra_similar:
                    if(porcentagem > 0.9):
                        terms.append("<i>"+similar+"</i>")

            
    dictMesh[d.find('.DescriptorUI').text] = {
        'ID' : d.find('.DescriptorUI').text,
        'name': d.find('.DescriptorName/String').text,
        'scope': scope,
        'terms': set(terms)
    }
                
    #break
len(dictMesh)

SyntaxError: invalid syntax (<ipython-input-135-3bd4462c9757>, line 25)

### Imprime Exemplo

In [109]:
#for i in dictMesh:
#    print (i)

### Salva Dicionario

In [130]:
import gzip, pickle

with gzip.open('dictMesh.dict.gz','wb') as fp:
    pickle.dump(dictMesh,fp)
    fp.close()

### Carrega Dicionario

In [132]:
import gzip, pickle

with gzip.open('dictMesh.dict.gz','rb') as fp:
    dictMesh = pickle.load(fp)
    fp.close()
    
dictMesh['D000001']

{'name': 'Calcimicina[Calcimycin]',
 'scope': 'An ionophorous, polyether antibiotic from Streptomyces chartreusensis. It binds and transports CALCIUM and other divalent cations across membranes and uncouples oxidative phosphorylation while inhibiting ATPase of rat liver mitochondria. The substance is used mostly as a biochemical tool to study the role of divalent cations in various biological systems.',
 'terms': {'A 23187',
  'A-23187',
  'A23187',
  'A23187, Antibiotic',
  'Antibiotic A23187',
  'Calcimicina',
  'Calcimycin'}}

In [134]:
palavra = 'abdomen'
for dui in dictMesh:
    d = dictMesh[dui]
    for t in d['terms']:
        if t.lower() == palavra.lower():
            print(dui, d['terms'])
            break

D000005 {'Abdome', '<i>adbome</i>', 'Abdômen', '<i>abdome</i>', 'Abdomen', '<i>abome</i>', '<i>abdomem</i>', '<i>adome</i>', '<i>abdomne</i>', '<i>adomen</i>', '<i>abdomen</i>', '<i>abd</i>', '<i>abm</i>', 'Abdomens', '<i>abomen</i>'}


In [122]:
termos=" "
confirma = False
evolucao = "abd"
strr = ""

#Verifica a lista para ver se a palavra esta no dicionario
cont = 0
for palavra in evolucao:
    for dui in dictMesh:
        d = dictMesh[dui]
        for t in d['terms']:
            if t.lower() == palavra.lower():
                evolucao[cont] = '<a href="#" data-ui="das" data-term="" data-scope="'+d['scope']+'">'+palavra+'</a>'
                cont +=1

    teste = dictMesh[dui]['terms']
    termos = ' '.join(teste)
evolucao = evolucao.replace('data-terms=""', 'data-terms="'+termos+'"')#pegar os termos da lista e por aqui
strr += ' '+evolucao

#print(termos)
termos

'Fibrinogen Deficiencies Deficiência de Fibrinogênio Afibrinogenemias Deficiency, Fibrinogen Fibrinogen Deficiency Afibrinogenemia'

In [123]:
d['terms']

{'Afibrinogenemia',
 'Afibrinogenemias',
 'Deficiency, Fibrinogen',
 'Deficiência de Fibrinogênio',
 'Fibrinogen Deficiencies',
 'Fibrinogen Deficiency'}

In [115]:
strr = "meu[nome]"
strr = strr.replace('[', ' [')
print(strr)


meu [nome]
